# 2023-07-10__AdEx_STA_Nto1

In [33]:
%run lib/neuron.py

In [34]:
μₓ = 4 * Hz
σ = sqrt(0.6)
μ = log(μₓ / Hz) - σ**2 / 2

1.0862943611198905

In [35]:
def Nto1(N = 6500):
    
    Ne = N * 4//5
    
    n = COBA_AdEx_neuron()
    
    rates = lognormal(μ, σ, N) * Hz;
    P = PoissonGroup(N, rates)
    
    Se = Synapses(P, n, on_pre="ge += we")
    Si = Synapses(P, n, on_pre="gi += wi")
    Se.connect("i < Ne")
    Si.connect("i >= Ne")
    
    M = StateMonitor(n, ["V"], record=[0])
    S = SpikeMonitor(n)
    SP = SpikeMonitor(P)
    
    objs = [n, P, Se, Si, M, S, SP]
    return *objs, Network(objs)

In [36]:
n, P, Se, Si, M, S, SP, net = Nto1()
net.store()

In [59]:
net.restore()
we = 14 * pS
wi = 4 * we
T = 10 * second
net.run(T, report='text')

Starting simulation at t=0. s for a duration of 10. s
2.963 s (29%) simulated in 10s, estimated 24s remaining.
6.449 s (64%) simulated in 20s, estimated 11s remaining.
9.5836 s (95%) simulated in 30s, estimated 1s remaining.
10. s (100%) simulated in 31s


In [60]:
ws = [14 * pS, 20 * pS]

T = 1 * second
data = []
for we in ws:
    wi = 4 * we
    for s in range(2):
        seed(s)
        net.restore()
        net.run(T, report='text')
        v = ceil_spikes(M, S)
        d = dict(
            we   = we,
            seed = s,
            median_Vm   = median(v),
            output_rate = S.num_spikes / T,
        )
        data.append(d)

Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 3s
Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 3s
Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 3s
Starting simulation at t=0. s for a duration of 1. s
1. s (100%) simulated in 2s


In [61]:
import pandas as pd

In [58]:
df = pd.DataFrame(data)

,we,seed,median_Vm,output_rate
0,14. pS,0,-53.67539721 mV,4. Hz
1,14. pS,1,-54.14036033 mV,4. Hz
2,20. pS,0,-53.75508981 mV,8. Hz
3,20. pS,1,-53.91442911 mV,8. Hz


In [14]:
from julia import Base

In [15]:
Base.sin(8)

0.9893582466233818